In [13]:
import pandas as pd
from haversine import haversine
import json
import datetime
import numpy as np
import zipcodes

In [14]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json"))

In [15]:
zip_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1,usecols=[0,2])
zip_DMA=zip_DMA.rename(columns={"CODE":"store_zip","NAME":"Market"})
zip_DMA=zip_DMA.drop_duplicates()

In [16]:
store_list_SFTP=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190401-134939-117.txt",dtype=str,sep="|")
print("5391" in store_list_SFTP['location_id'].tolist())
print("5215" in store_list_SFTP['location_id'].tolist())

SOTF_list=['5391','5215']

store_list_SFTP=store_list_SFTP[store_list_SFTP['location_id'].isin(SOTF_list)]
store_list_SFTP=store_list_SFTP[['location_id','zip_cd','latitude_meas','longitude_meas']].rename(columns={"location_id":"Store ID","zip_cd":"store_zip","latitude_meas":"Latitude_meas","longitude_meas":"Longitude_meas"})
store_list_SFTP['store_zip']=store_list_SFTP['store_zip'].apply(lambda x: x.split("-")[0].zfill(5))
store_list_SFTP['Latitude_meas']=store_list_SFTP['Latitude_meas'].astype(float)
store_list_SFTP['Longitude_meas']=store_list_SFTP['Longitude_meas'].astype(float)

store_list_SFTP['G.O.']="2019-05-24"
store_list_SFTP

False
True


,Store ID,store_zip,Latitude_meas,Longitude_meas,G.O.
1233,5215,32405,30.187841,-85.670084,2019-05-24


In [17]:
store_level_zips=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",
                             dtype=str,usecols=['location_id','zip','revenue_flag'])
store_level_zips=store_level_zips[store_level_zips['location_id'].isin(SOTF_list)]
print(store_level_zips['location_id'].unique())
store_level_zips=store_level_zips[store_level_zips['revenue_flag'].isin(['P','S'])]
store_level_zips=store_level_zips.rename(columns={"location_id":"Store ID","revenue_flag":'Revenue_flag'})

['5215']


In [20]:
output_0517=pd.merge(store_list_SFTP,store_level_zips,on="Store ID",how="left")
output_0517=output_0517[['Store ID','G.O.','Longitude_meas','Latitude_meas','Revenue_flag','zip','store_zip']]

In [21]:
output_0517['Distance (mile)']=np.nan
for i in range(len(output_0517)):
    zip_cd=output_0517['zip'][i]
    store_center=(output_0517['Latitude_meas'][i],output_0517['Longitude_meas'][i])
    if zip_cd in zip_centers.keys():
        zip_center=zip_centers[zip_cd]
        dist=haversine(store_center,zip_center,miles=True)
        output_0517['Distance (mile)'][i]=dist
    else:
        try:
            zip_center=[zipcodes.matching(zip_cd)[0]['lat'],zipcodes.matching(zip_cd)[0]['long']]
            dist=haversine(store_center,zip_center,miles=True)
            output_0517['Distance (mile)'][i]=dist
        except:
            output_0517['Distance (mile)'][i]=np.nan
            
output_0517=output_0517[(output_0517['Revenue_flag']!="S") | (output_0517['Distance (mile)']<=20)]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [8]:
store_location_dict={}
# address in excel: 1251 BURKEMONT AVENUE	MAGNOLIA PLAZA	MORGANTON	NC	28655
# address googled: 1251 Burkemont Ave, Morganton, NC 28655
store_location_dict.update({"0":{"Store ID":"5391",
                            "store_zip":"28655",
                            "Latitude_meas":35.715747,
                            "Longitude_meas":-81.6964109,
                            "G.O.":"2019-05-17"}})

store_list_5391=pd.DataFrame(store_location_dict).T

store_list_5391=store_list_5391[store_list_SFTP.columns.tolist()]

In [9]:
store_number=store_list_5391['Store ID'].tolist()[0]    
df_5391=pd.DataFrame({"Store ID":store_number,"zip":store_list_5391['store_zip'].tolist()[0],'Revenue_flag':"zip_in_10","Distance (mile)":"store_zip"},index=[0])


store_centers=[store_list_5391['Latitude_meas'].tolist()[0],store_list_5391['Longitude_meas'].tolist()[0]]

for zip_cd in zip_centers.keys():
    zip_center=zip_centers[zip_cd]    
    dist=haversine(store_centers,zip_center,miles=True)
    i_counter=1
    if dist<=10:
        df=pd.DataFrame({"Store ID":store_number,"zip":zip_cd,'Revenue_flag':"zip_in_10","Distance (mile)":dist},index=[i_counter])
        i_counter+=1
        df_5391=df_5391.append(df)

In [10]:
output_0524=pd.merge(store_list_5391,df_5391,on="Store ID",how="left")
output_0524=output_0524[output_0517.columns.tolist()]

In [11]:
import os
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Zips_2_stores_SOTF_20190502'

In [12]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Zips_2_stores_SOTF_20190502/BL_zips_for_2_SOTF_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

output_0517.to_excel(writer,"BL_5.17 Zips_1 Store",index=False)
output_0524.to_excel(writer,"BL_5.24 Zips_1 Store",index=False)

writer.save()